# Create and Test Model Serving (nuclio) Function
The following notebook demonstrates how to write an inferencing server, test it, and turn it into an auto-scaling nuclio function

## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that dont need to be copied to the function

In [1]:
# nuclio: ignore
import nuclio
import random
import matplotlib.pyplot as plt

In [2]:
%%nuclio env
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
version = 1.0

%nuclio: setting 'IMAGE_WIDTH' environment variable
%nuclio: setting 'IMAGE_HEIGHT' environment variable
%nuclio: setting 'version' environment variable


In [18]:
%nuclio env -c MODEL_PATH=/model/
%nuclio env -l MODEL_PATH=./model/

%nuclio: setting 'MODEL_PATH' environment variable


In [4]:
%%nuclio cmd -c
pip install git+https://github.com/fchollet/keras
pip install tensorflow 
pip install numpy
pip install requests
pip install pillow

In [5]:
%%nuclio config 
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


In [6]:
%nuclio mount /model ~/demos/cats_dogs/model

mounting volume path /model as ~/demos/cats_dogs/model


# Create and Load Model and setup handeler 

In [9]:
import numpy as np 
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img
import json
import requests

import os
from os import environ, path
from tempfile import mktemp

Using TensorFlow backend.


In [12]:
model_file = environ['MODEL_PATH'] + 'cats_dogs.hd5'
prediction_map_file = environ['MODEL_PATH'] + 'prediction_classes_map.json'

# set image parameters
IMAGE_WIDTH = int(environ['IMAGE_WIDTH'])
IMAGE_HEIGHT = int(environ['IMAGE_HEIGHT'])

# load model
def init_context(context): 
    context.model = load_model(model_file)
    with open(prediction_map_file, 'r') as f:
        context.prediction_map = json.load(f)

In [13]:
def download_file(context, url, target_path):
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        with open(target_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

    context.logger.info_with('Downloaded file',url=url)

In [14]:
def handler(context, event):
    tmp_file = mktemp()
    image_url = event.body.decode('utf-8').strip()
    download_file(context, image_url, tmp_file)
    
    img = load_img(tmp_file, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    predicted_probability = context.model.predict_proba(images, batch_size=10)
    predicted_class = list(zip(predicted_probability, map(lambda x: '1' if x >= 0.5 else '0', predicted_probability)))
    actual_class = [(context.prediction_map[x[1]],x[0][0]) for x in predicted_class]   
    os.remove(tmp_file)
    return str(actual_class)

# Trigger the function

In [ ]:
# nuclio: ignore
init_context(context)

In [ ]:
# nuclio: ignore
# Select sample for the test (both local path for test and the URL from S3)
DATA_LOCATION = "./cats_and_dogs_filtered/"
sample = random.choice(os.listdir(DATA_LOCATION+"/cats_n_dogs"))
image_local = DATA_LOCATION + "cats_n_dogs/"+sample #temp to get a file 
image_url = 'https://s3.amazonaws.com/iguazio-sample-data/images/catanddog/' + sample 

# show the image
img = load_img(image_local, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
plt.imshow(img)

event = nuclio.Event(body=bytes(image_url, 'utf-8'))
output = handler(context, event)
print(f'output:\nclass:{output[0][0]},\tdog-probability:{output[0][1]:.2f}')

In [ ]:
%nuclio show

In [23]:
%nuclio deploy -n cats-dogs -p ai -c

%nuclio: ['deploy', '-n', 'cats-dogs', '-p', 'ai', '-c', '/User/demos/cats_dogs/infer.ipynb']
%nuclio: [nuclio.deploy] 2019-03-17 23:59:02,689 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-03-17 23:59:04,713 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-03-17 23:59:04,713 (info) Build complete
%nuclio: [nuclio.deploy] 2019-03-17 23:59:09,764 done updating cats-dogs, function address: 18.184.175.114:31731
%nuclio: function deployed


In [24]:
!curl -X POST -d "https://s3.amazonaws.com/iguazio-sample-data/images/catanddog/cat.2004.jpg" 18.184.175.114:31731

[('cat', 0.0)]